ETL Project- Meghan, Nabila, Gabriel

Extract: For this portion, we extracted data from kaggle.com from two sources. One was from the New York Times of Covid-19 case information by state, and the USA COVID-19 vaccination data by state. These were both in CSV format. 


Transform: First we loaded the data into jupyter notebook, and cleaned it up. We removed the null data points. Then we went into pgAdmin and made a new database for this information. We created tables for each CSV file, and joined the tables together. 

Load: We went back to our jupyter notebook and connected the data to the local SQL database. Then we used pandas to load our CSV converted DataFrame into the database. After we connected them together, we combined our collections and showed it in a dataframe. The final table has all the relevant columns for comparing COVID vaccinations and cases by date and state. We chose this method because it was the most efficient process for the data to be set up and analyzed.


In [1]:
# Import Dependencies

from sklearn import datasets
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Import data file
us_state_vaccinations_df = pd.read_csv('us_state_vaccinations.csv')
us_state_vaccinations_df.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


In [3]:
# Remove null rows (if needed)
us_state_vaccinations_df.dropna(inplace=True)
us_state_vaccinations_df.shape

us_state_vaccinations_df

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
7,2021-01-19,Alabama,130795.0,444650.0,114319.0,0.33,2.67,16346.0,2.33,9.07,7557.0,7523.0,1534.0,0.294
8,2021-01-20,Alabama,139200.0,483275.0,121113.0,0.37,2.84,17956.0,2.47,9.86,8405.0,7880.0,1607.0,0.288
9,2021-01-21,Alabama,165919.0,493125.0,144429.0,0.44,3.38,21345.0,2.95,10.06,26719.0,10517.0,2145.0,0.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,2021-03-08,Wyoming,185580.0,236065.0,114824.0,11.79,32.07,68219.0,19.84,40.79,7545.0,4120.0,7119.0,0.786
3919,2021-03-09,Wyoming,185813.0,246125.0,115046.0,11.80,32.11,68289.0,19.88,42.53,233.0,4148.0,7167.0,0.755
3920,2021-03-10,Wyoming,190180.0,259495.0,116799.0,12.25,32.86,70919.0,20.18,44.84,4367.0,3895.0,6730.0,0.733
3921,2021-03-11,Wyoming,190570.0,262995.0,116948.0,12.30,32.93,71215.0,20.21,45.44,390.0,3911.0,6758.0,0.725


In [4]:
us_state_vaccinations_df = us_state_vaccinations_df[['date', 'location', 'daily_vaccinations']].copy()
us_state_vaccinations_df.head()

,date,location,daily_vaccinations
1,2021-01-13,Alabama,5906.0
3,2021-01-15,Alabama,7478.0
7,2021-01-19,Alabama,7523.0
8,2021-01-20,Alabama,7880.0
9,2021-01-21,Alabama,10517.0


In [5]:
# Import data file
States_df = pd.read_csv('States.csv')
States_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
# Remove null rows (if needed)
States_df.dropna(inplace=True)
States_df.shape

States_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20414,2021-03-08,Virginia,51,586592,9683
20415,2021-03-08,Washington,53,348485,5129
20416,2021-03-08,West Virginia,54,133627,2325
20417,2021-03-08,Wisconsin,55,621842,7111


In [7]:
#Picked out relevant columns 

States_df = States_df[['date', 'state', 'cases']].copy()
States_df.head()

,date,state,cases
0,2020-01-21,Washington,1
1,2020-01-22,Washington,1
2,2020-01-23,Washington,1
3,2020-01-24,Illinois,1
4,2020-01-24,Washington,1


In [8]:
#Connect to local database

rds_connection_string = "postgres:farooqi3792@localhost:5432/ETL_project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [9]:
#Check for tables
engine.table_names()

['vaccinations', 'cases']

In [12]:
# Use pandas to load csv converted DataFrame into database
us_state_vaccinations_df.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)

In [13]:
# Use pandas to load csv converted DataFrame into database
States_df.to_sql(name='cases', con=engine, if_exists='append', index=False)

In [14]:
#confirm data added for vaccinations by state
pd.read_sql_query('select * from vaccinations', con=engine).head()

,date,location,daily_vaccinations
0,2021-01-13,Alabama,5906
1,2021-01-15,Alabama,7478
2,2021-01-19,Alabama,7523
3,2021-01-20,Alabama,7880
4,2021-01-21,Alabama,10517


In [15]:
#confirm data added for Cases by state
pd.read_sql_query('select * from cases', con=engine).head()

,date,state,cases
0,2020-01-21,Washington,1
1,2020-01-22,Washington,1
2,2020-01-23,Washington,1
3,2020-01-24,Illinois,1
4,2020-01-24,Washington,1


In [16]:
#Combine datasets together 
combined_df = pd.read_sql_query('SELECT * FROM vaccinations INNER JOIN cases ON vaccinations.date = cases.date AND vaccinations.location = cases.state', con=engine)
combined_df

,date,location,daily_vaccinations,date,state,cases
0,2021-01-13,Alabama,5906,2021-01-13,Alabama,410995
1,2021-01-13,Alaska,2842,2021-01-13,Alaska,50526
2,2021-01-13,Arizona,13390,2021-01-13,Arizona,643683
3,2021-01-13,Arkansas,53975,2021-01-13,Arkansas,262020
4,2021-01-13,California,75188,2021-01-13,California,2857587
...,...,...,...,...,...,...
2680,2021-03-08,Virginia,49929,2021-03-08,Virginia,586592
2681,2021-03-08,Washington,59036,2021-03-08,Washington,348485
2682,2021-03-08,West Virginia,9765,2021-03-08,West Virginia,133627
2683,2021-03-08,Wisconsin,43428,2021-03-08,Wisconsin,621842


In [17]:
#Picked out relevant columns 
combined_df = combined_df[['date', 'location', 'daily_vaccinations','cases']].copy()
combined_df.head()

,date,date,location,daily_vaccinations,cases
0,2021-01-13,2021-01-13,Alabama,5906,410995
1,2021-01-13,2021-01-13,Alaska,2842,50526
2,2021-01-13,2021-01-13,Arizona,13390,643683
3,2021-01-13,2021-01-13,Arkansas,53975,262020
4,2021-01-13,2021-01-13,California,75188,2857587
